# How GDS works
At a high-level, GDS works by transforming and loading data into an in-memory format that is optimized for high-performance graph analytics. GDS provides graph algorithms, feature engineering, and machine learning methods to execute on this in-memory graph format.


Below is diagram illustrating the general workflow in GDS, which breaks out into 3 high-level steps


<img title="a title" alt="Alt text" src="./images/gds-workflow.png">


1. **Read and Load the Graph**: GDS needs to read data from the Neo4j database, transform it, and load it into an in-memory graph. In GDS we refer to this process as **projecting** a graph and refer to the in-memory graph as a **graph projection**. GDS can hold multiple graph projections at once and they are managed by a component called the **Graph Catalog**. 

2. **Execute Algorithms**: This includes classic graph algorithms such as **centrality**, **community detection**, **path finding**, etc. It also includes embeddings, a form of robust graph **feature engineering**, as well as machine learning pipelines.

3. **Store Results**: There are a few things you may want to do with the output/result of graph algorithms. GDS enables you to write results back to the database, export to disk in csv format, or stream results into another application or downstream workflow.

GDS uses multiple CPU cores for graph projections, algorithms, and writing results. This allows GDS to parallelize its computations and significantly speed up processing time

<img title="a title" alt="Alt text" src="./images/neo4j-memory.management.png"  width="50%">

Of the above, two main types of memory can be allocated in configuration:

- **Heap Space**: Used for **storing in-memory graphs**, executing GDS algorithms, query execution, and transaction state

- **Page Cache**: Used for **indexes and to cache the Neo4j data stored on disk**. Improves performance for querying the database and projecting graphs.


# Graph Catalog

The graph catalog is a concept that **allows you to manage graph projections in GDS**. This includes

- creating (a.k.a projecting) graphs
- viewing details about graphs
- dropping graph projections
- exporting graph projections
- writing graph projection properties back to the database

## list graph projections
For example, we can list the graph projections that currently exist in our database with the below command.
```
CALL gds.graph.list()
```

In the recommendations graph, **we can create a projection from the `Actor` and `Movie` nodes and the `ACTED_IN` relationship with the below command.**

```
CALL gds.graph.project('my-graph-projection', ['Actor','Movie'], 'ACTED_IN')
```

If we now list graphs again we should see information on the graph we just made

```
CALL gds.graph.list() YIELD graphName, nodeCount, relationshipCount, schema
```

|"graphName" |	"nodeCount" |	"relationshipCount" |	"schema"|
|-----------------|--------------|------------|------|
|"my-graph-projection" | 24568 | 35910 | {"relationships":{"ACTED_IN":{}},"nodes":{"Movie":{},"Actor":{}}} |


## Dropping Graphs
Projected graphs take up space in memory so once we are done working with a graph projection it is smart to remove it. We can do this with the drop command below:

```
CALL gds.graph.drop('my-graph-projection')
```
## Exporting Graphs
export data from a graph projection after performing graph algorithms and other analytics. For example, you may want to:

- export graph features for training a machine learning model in another environment
- create separate analytical views for downstream analytics and/or sharing with colleagues.
- produce snapshots of analytical results and persist to the filesystem

The graph catalog has two methods for export:

1. `gds.graph.export` to export a graph into a new database - effectively copying the projection into a separate Neo4j database

2. `gds.beta.graph.export.csv` to export a graph to csv files

# Projections
**There are 2 primary types of projections in GDS, native projections and cypher projections.** 
- **native projections** are optimized for `efficiency and performance` to support graph data science at scale.
- **Cypher projections** are optimized for `flexibility and customization` to support exploratory analysis, experimentation, and smaller graph projections.

## Native Projections

When you call `gds.graph.project()` you are using a native projection. Native projections provide the best performance by reading from the Neo4j store files directly. We recommend them for both development and production phases.

### Basic Syntax
**The native projection takes three mandatory arguments**: `graphName`, `nodeProjection` and `relationshipProjection`. In addition, the optional configuration parameter allows us to further configure the graph creation.

|Name	                    |Type	                |Optional	|Description                                               |
|-------------------------|---------------------|---------|----------------------------------------------------------|
| `graphName`             | String              | no      | The name under which the graph is stored in the catalog. |
| `nodeProjection`        | String, List or Map | no      | The configuration for projecting nodes.                  | 
| `relationshipProjection`| String, List or Map | no      | The configuration for projecting relationships.          | 
| `configuration`         | Map                 | yes     | Additional parameters to configure the native projection.|

```
CALL gds.graph.project('native-proj',['User', 'Movie'], ['RATED']);
```

**The wildcard character** `'*'` can be used to **include all nodes and/or relationships in the database**. The below projections all nodes and relationships.

```
CALL gds.graph.project('native-proj','*', '*');
```
#### change relationship orientation
Native projections allow you to **change the relationship orientation** as well.

- A `directed relationship` is **non-symmetrical**. It **goes from a source node to a target node**
- An `undirected relationship` is **symmetric with no directional character**, it is simply between two nodes instead of having a source and target.

**Every relationship in the neo4j database is directed by design.** However, some graph algorithms are designed to work on undirected relationships.
To accommodate this there are **three orientation options** we can apply to relationship types in the relationshipProjection:

- `NATURAL`: same direction as in the database (default)
- `REVERSE`: opposite direction as in the database
- `UNDIRECTED`: undirected

##### Example
```
CALL gds.graph.drop('native-proj', false);

//replace with a project that has reversed relationship orientation
CALL gds.graph.project(
    'native-proj',
    ['User', 'Movie'],
    {RATED_BY: {type: 'RATED', orientation: 'REVERSE'}}
);

CALL gds.degree.mutate('native-proj', {mutateProperty: 'ratingCount'});
```

Now when we use the **degree algorithm** we will get the rating counts we need.

```
CALL gds.graph.nodeProperty.stream('native-proj','ratingCount', ['Movie'])
YIELD nodeId, propertyValue
RETURN gds.util.asNode(nodeId).title AS movieTitle, propertyValue AS ratingCount
ORDER BY movieTitle DESCENDING LIMIT 5
```

#### Including Node and Relationship Properties
**Node and relationship properties may be useful to consider in graph analytics**. They can be used as **weights** in graph algorithms and features for machine learning.

example of including multiple movie node properties and the rating relationship property.

```
CALL gds.graph.drop('native-proj', false);

CALL gds.graph.project(
    'native-proj',
    ['User', 'Movie'],
    {RATED: {orientation: 'UNDIRECTED'}},
    {
        nodeProperties:{
            revenue: {defaultValue: 0}, // (1)
            budget: {defaultValue: 0},
            runtime: {defaultValue: 0}
        },
        relationshipProperties: ['rating'] // (2)
    }
);
```

#### Parallel Relationship Aggregations
The Neo4j database allows you to **store multiple relationships of the same type and direction between two nodes**. These are colloquially known as `parallel relationships`. Sometimes you will want to **aggregate these parallel relationships** into a **single relationship** in preparation for running graph algorithms or machine learning. 
Other times we may want to **weight the connection between two nodes higher if more parallel relationships exists**.

example of `aggregating relationships` without any properties

```
CALL gds.graph.project(
  'user-proj',
  ['User'],
  {
    SENT_MONEY_TO: { aggregation: 'SINGLE' }
  }
);
```
We can `create a property with the count of the relationships` as well - like so:


```
CALL gds.graph.project(
  'user-proj',
  ['User'],
  {
    SENT_MONEY_TO: {
      properties: {
        numberOfTransactions: {
          // the wildcard '*' is a placeholder, signaling that
          // the value of the relationship property is derived
          // and not based on Neo4j property.
          property: '*',
          aggregation: 'COUNT'
        }
      }
    }
  }
);
```

We can also take the **sum**, **min** or **max** of relationship properties during aggregation. Below is an example with sum.


## Cypher Projections

While the native projection is scalable and fast, its filtering and aggregation capabilities aren’t as flexible as Cypher. The Cypher projection, as its name implies, **uses Cypher to define the projection pattern**, and as such, enables **more flexibility**.

While Cypher projections offer more flexibility and customization, they have a **diminished focus on performance** relative to native projections and as a result **won’t perform as quickly or as well on larger graphs**.



### Basic Syntax
A Cypher projection takes three mandatory arguments: `graphName`, `nodeQuery`, and `relationshipQuery`. In addition, the optional `configuration` parameter allows us to further configure graph creation.


|Name	            |Optional	|Description                                               |
|-------------------|-----------|----------------------------------------------------------|
|  `graphName`        |  no       |  The name under which the graph is stored in the catalog.|  
|  `nodeQuery`        |  no       |  Cypher statement to project nodes. The query result must contain an `id` column. Optionally, a `labels` column can be specified to represent node labels. Additional columns are interpreted as properties.|  
|  `relationshipQuery`|  no       |  Cypher statement to project relationships. The query result must contain source and target columns. Optionally, a type column can be specified to represent relationship type. Additional columns are interpreted as properties.|  
|  `configuration`    |  yes      |  Additional parameters to configure the Cypher projection.|



```
CALL gds.graph.project.cypher(
  /* The name under which the graph is stored in the catalog */
  'proj-cypher',

  /* Cypher statement to project nodes */
  'MATCH (a:Actor) RETURN id(a) AS id, labels(a) AS labels',

  /* Cypher statement to project relationships */
  'MATCH (a1:Actor)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(a2)
   WHERE m.year >= 1990 AND m.revenue >= 1000000
   RETURN id(a1) AS source , id(a2) AS target, count(*) AS actedWithCount, "ACTED_WITH" AS type'
);
```
Once that is done **we can apply degree centrality** like we did last lesson. Except **we will weight the degree centrality** by `actedWithCount` property and also directly stream the top 10 results back. This counts how many times the actor has acted with other actors in recent, high grossing movies.

```
CALL gds.degree.stream('proj-cypher',{relationshipWeightProperty: 'actedWithCount'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score
ORDER BY score DESC LIMIT 10
```

# Graph Algorithm

## Tiers
GDS algorithms are classified into three tiers: **alpha**, **beta**, and **production**.

- `Production-quality`: Indicates that the algorithm has been tested in regard to stability and scalability. Algorithms in this tier are prefixed with gds.<algorithm>.
- `Beta`: Indicates that the algorithm is a candidate for the production-quality tier. Algorithms in this tier are prefixed with gds.beta.<algorithm>.
- `Alpha`: Indicates that the algorithm is experimental and might be changed or removed at any time. Algorithms in this tier are prefixed with gds.alpha.<algorithm>.

## Execution Modes
GDS algorithms have **4 executions modes which determine how the results of the algorithm are handled**.

1. `stream`: Returns the result of the algorithm as a stream of records.
2. `stats`: Returns a single record of summary statistics, but does not write to the Neo4j database or modify any data.
3. `mutate`: Writes the results of the algorithm to the in-memory graph projection and returns a single record of summary statistics.
4. `write`: Writes the results of the algorithm back the Neo4j database and returns a single record of summary statistics.

### Overall Algorithm Syntax
Putting the above together, **all GDS algorithms follow the below syntax**:

```
CALL gds[.<tier>].<algorithm>.<execution-mode>[.<estimate>](
	graphName: STRING,
	configuration: MAP
)
```

## Centrality
**Centrality algorithms are used to determine the importance of distinct nodes in a graph.**

Common use cases of centrality include:

- `Recommendations`: Identify and recommend the most influential or popular items in your content or product offering catalog
- `Supply chain analytics`: find the most critical node in your supply chain, whether it be a supplier in a network, a raw material that is part of a manufactured product, or a port in a route
- `Fraud & Anomaly Detection`: Find users with many shared identifiers or who otherwise act as a bridge between many communities

### Degree Centrality
**It counts the number of relationships a node has.** In the GDS implementation, we specifically calculate `out-degree centrality` which is the **count of outgoing relationships from a node.**

First create the graph projection.

```
CALL gds.graph.project('proj', ['Actor','Movie'], 'ACTED_IN');
```

Then stream the degree centrality.

```
//get top 5 most prolific actors (those in the most movies)
//using degree centrality which counts number of `ACTED_IN` relationships
CALL gds.degree.stream('proj')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS actorName, score AS numberOfMoviesActedIn
ORDER BY numberOfMoviesActedIn DESCENDING, actorName LIMIT 5
```

### Page Rank
**PageRank is a good algorithm for measuring the influence of nodes in a directed graph, particularly where the relationships imply some form of flow of movement such as in payment networks, supply chain and logistics, communications, routing, and graphs of website and links.**

**PageRank estimates the importance of a node by counting the number of incoming relationships from neighboring nodes weighted by the importance and out-degree centrality of those neighbors.** 

The underlying assumption is that **more important nodes are likely to have proportionately more incoming relationships from other important nodes**.

**First**, create the graph projection. We can use a Cypher projection in this case to obtain a graph where we have `(Person)-[:DIRECTED_ACTOR]->(Person)`. this graph can be traversed to understand the influence across directors and actors.

```
//drop last graph projection
CALL gds.graph.drop('proj', false);

//create Cypher projection for network of people directing actors
//filter to recent high grossing movies
CALL gds.graph.project.cypher(
  'proj',
  'MATCH (a:Person) RETURN id(a) AS id, labels(a) AS labels',
  'MATCH (a1:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a2)
   WHERE m.year >= 1990 AND m.revenue >= 10000000
   RETURN id(a1) AS source , id(a2) AS target, count(*) AS actedWithCount,
    "DIRECTED_ACTOR" AS type'
);
```

**Next** stream `PageRank` to find the top 5 most influential people in director-actor network.

```
CALL gds.pageRank.stream('proj')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS personName, score AS influence
ORDER BY influence DESCENDING, personName LIMIT 5
```


### Others Centrality Algorithms

- `Betweenness Centrality`: Measures the **extent to which a node stands between the other nodes in a graph**. It is often used to find nodes that serve as a **bridge** from one part of a graph to another.
- `Eigenvector Centrality`: Measures the **transitive influence of nodes**. Similar to PageRank, but works only on the largest eigenvector of the adjacency matrix so does not converge in the same way and tends to more strongly favor high degree nodes. It can be more appropriate in certain use cases, particularly those with undirected relationships.
- `Article Rank`: A variant of PageRank which assumes that relationships originating from low-degree nodes have a higher influence than relationships from high-degree nodes.


## Path Finding
Path finding algorithms find the **shortest path** between two or more nodes or evaluate the availability and quality of paths.

- **Supply chain analytics**: Identifying the **fastest path between an origin and a destination** or between a raw material and a finished product
- **Customer Journey**: Analyzing the **events that make up a customer’s experience**. In healthcare for example, this can be the experience of an in-patient from admission to discharge.


### Dijkstra Source-Target Shortest Path

**It computes the shortest path between a source and a target node.** Like many other path finding algorithms in GDS, **Dijkstra supports weighted relationships to account for distance or another cost property when comparing paths.**

**First**, create the graph projection.

```
CALL gds.graph.project('proj',
    ['Person','Movie'],
    {
        ACTED_IN:{orientation:'UNDIRECTED'},
        DIRECTED:{orientation:'UNDIRECTED'}
    }
);
```

Then we can run **Dijkstra’s shortest path**.

```
MATCH (a:Actor)
WHERE a.name IN ['Kevin Bacon', 'Denzel Washington']
WITH collect(id(a)) AS nodeIds
CALL gds.shortestPath.dijkstra.stream('proj', {sourceNode:nodeIds[0], TargetNode:nodeIds[1]})
YIELD sourceNode, targetNode, path
RETURN gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    nodes(path) as path;
```


### Other Path Finding Algorithms
Other GDS production tier Path Finding algorithms can be split into a few different subcategories that are listed below:

**Shortest path between two nodes:**

- `A* Shortest Path`: An extension of Dijkstra that uses a heuristic function to speed up computation.
- `Yen’s Algorithm Shortest Path`: An extension of Dijkstra that allows you to find multiple, the **top k**, shortest paths.

**Shortest path between a source node and multiple other target nodes:**

- `Dijkstra Single-Source Shortest Path`: Dijkstra implementation for shortest path between one source and multiple targets.
- `Delta-Stepping Single-Source Shortest Path`: Parallelized shortest path computation. Computes faster than Dijkstra single-source shortest Path but uses more memory.

**General path search between a source node and multiple other target nodes:**

- `Breadth First Search`: Searches paths in order of increasing distance from the source node on each iteration.
- `Depth First Search`: Searches as far as possible along a single multi-hop path on each iteration.

## Community Detection
**Community detection algorithms are used to evaluate how groups of nodes may be clustered or partitioned in the graph.**

Common use cases of community detection include:

- `Fraud detection`: Finding fraud rings by identifying accounts that have frequent suspicious transactions and/or share identifiers between one another.
- `Customer 360`: Disambiguating multiple records and interactions into a single customer profile so an organization has an aggregated source of truth for each customer.
- `Market segmentation`: dividing a target market into approachable subgroups based on priorities, behaviors, interests, and other criteria.


### Louvain Community Detection

**Louvain maximizes a modularity score for each community, where the modularity quantifies the quality of an assignment of nodes to communities. This means evaluating how much more densely connected the nodes within a community are, compared to how connected they would be in a random network.**

Louvain optimizes this modularity with a hierarchical clustering approach that recursively merges communities together. 

An additional important consideration is that **Louvain is a stochastic algorithm**. As such, the community assignments may change a bit when re-run.

Louvain includes a `seedProperty` parameter which can be used to assign initial community ids and help with consistency between runs.

**First** create a graph projection with movies, actors, and directors. Project the relationships with an UNDIRECTED orientation as that works best with the Louvain algorithm.

```
CALL gds.graph.project('proj', ['Movie', 'Person'], {
    ACTED_IN:{orientation:'UNDIRECTED'},
    DIRECTED:{orientation:'UNDIRECTED'}
});
```

**Then we can run Louvain**. Here we will run Louvain in mutate mode to save community Ids and return high level statistics on the community counts, distribution, modularity score, and information for how Louvain processed the graph.

```
CALL gds.louvain.mutate('proj', {mutateProperty:'communityId'})
```

We can verify the `communityId` node properties in the projection with a stream operation.

```
CALL gds.graph.nodeProperty.stream('proj','communityId', ['Person'])
YIELD nodeId, propertyValue
WITH gds.util.asNode(nodeId) AS n, propertyValue AS communityId
WHERE n:Person
RETURN n.name, communityId LIMIT 10
```

### Other Community Detection Algorithms
Below are some of the **other production tier community detection algorithms**. 

- **Label Propagation**: Similar intent as Louvain. Fast algorithm that parallelizes well. Great for large graphs.
- **Weakly Connected Components (WCC)**: Partitions the graph into sets of connected nodes such that
  - Every node is reachable from any other node in the same set
  - No path exists between nodes from different sets
- **Triangle Count**: Counts the number of triangles for each node. Can be used to detect the cohesiveness of communities and stability of the graph.
- **Local Clustering Coefficient**: Computes the local clustering coefficient for each node in the graph which is an indicator for how the node clusters with its neighbors.


## Node Embedding

**The goal of node embedding is to compute low-dimensional vector representations of nodes such that similarity between vectors (eg. dot product) approximates similarity between nodes in the original graph.**

The embedding thus **took the structure from the graph, the n-dimensional adjacency matrix, and approximated it in 2-dimensional vectors for each node**. Of course, in real-world problems node embeddings will usually be larger than 2 dimensions, often ending up in the hundreds or larger, especially when applied to bigger graphs with millions or billions of nodes.

<img title="a title" alt="Alt text" src="./images/node-embeddings-1.png"  width="50%">


Common workflows include:

- `Exploratory Data Analysis (EDA)` such as visualizing the embeddings in a TSNE plot to better understand the graph structure and potential clusters of nodes
- `Similarity Measurements`: Node embedding allows you to scale similarity inferences in large graphs using K Nearest Neighbor (KNN) or other techniques. This can be useful for scaling memory based recommendation systems, such as variations of collaborative filtering. It can also be used for semi-supervised techniques in areas like fraud detection, where, for example, we may want to generate leads that are similar to a group of known fraudulent entities.
- `Features for Machine Learning`: Node embedding vectors naturally plug in as features for a variety of machine learning problems. For example, in a graph of user purchases for on online retailer, we could use embeddings to train a machine learning model to predict what products a user may be interested in buying next.

### FastRP

GDS offers a custom implementation of a node embedding technique called **Fast Random Projection**, or **FastRP** for short. FastRP leverages probabilistic sampling techniques to generate sparse representations of the graph allowing for extremely fast calculation of embedding vectors that are comparative in quality to those produced with traditional random walk and neural net techniques such as **Node2vec** and **GraphSage**.

### Other Node Embedding Algorithms
GDS has also implemented **Node2Vec**, which computes a vector representation of a node based on random walks in the graph, and **GraphSage**, which is an inductive modeling approach for computing node embeddings using node properties and graph structure.


## Similarity
**Similarity algorithms**, as the name implies, **are used to infer similarity between pairs of nodes.** 

 **When similar node pairs are identified according to the user specified metric and threshold, a relationship with a similarity score property is drawn between the pair**. 

Common use cases for similarity include:

- **Fraud detection**: finding potential fraud user accounts by analyzing how similar a set of new user accounts is to flagged accounts
- **Recommendation Systems**: In an online retail store, identifying items that pair to the one currently being viewed by a user to inform impressions and increase rate of purchase
- **Entity Resolution**: Identify nodes that are similar to one another based on activity or identifying information in the graph

GDS has **two** primary similarity algorithms:

- `Node Similarity`: Determines similarity between nodes **based on the relative proportion of shared neighboring nodes in the graph**. Node Similarity is a good choice where explainability is important, and you can narrow down the universe of comparisons to a subset of your data. Examples of narrowing down include focusing on just single communities, newly added nodes, or nodes within a specific proximity to a subgraph of interest.
- `K-Nearest Neighbor (KNN)`: Determines similarity **based off node properties**. The GDS KNN implementation can scale well for global inference over large graphs when tuned appropriately. **It can be used in conjunction with embeddings and other graph algorithms to determine the similarity between nodes based on proximity in the graph, node properties, community structure, importance/centrality, etc.**

Both `Node Similarity` and `KNN` **provide choices between different similarity metrics**. Node Similarity has choices between `Jaccard` and `Overlap similarity`. KNN choice of metric is driven by the node property types. List of integers are subject to Jaccard and Overlap, list of floating point numbers to `Cosine Similarity`, `Pearson`, and `Euclidean`.


**Comparing every node to every other node in the graph is a computationally expensive task of roughly `O(n^2)` complexity.**

**Node Similarity** has a `degreeCutOff` parameter for nodes which allows you to **set a lower limit on the degree centrality** for nodes to be selected.

**KNN** has various parameters that can be tuned to affect the speed vs completeness trade-off of node comparisons, including sampling rate, initial sampler methodology, random join counts between iteration, and a few others.

For similarity comparisons we may also want to **control the number of results returned** to only consider the most relevant node pairs. Both Node Similarity and KNN have a `topK` parameter to **limit the number similarity comparisons returned per node**.

### Example

```
CALL gds.graph.project('proj', ['Movie', 'Person'], {
    ACTED_IN:{orientation:'UNDIRECTED'},
    DIRECTED:{orientation:'UNDIRECTED'}
});
```

**We will then run FastRP** like in the last lesson except in mutate mode so the embeddings will be saved in the projection.

```
CALL gds.fastRP.mutate('proj',  {
    embeddingDimension:64,
    randomSeed:7474,
    mutateProperty:'embedding'
})
```

**After that we can run similarity**. We will use the default **cosine** metric. For purposes of demonstration, I will limit **topK** to one so we can see the top pairs for each node.

```
CALL gds.knn.stream('proj', {nodeLabels:['Person'], nodeProperties:['embedding'], topK:1})
YIELD  node1, node2, similarity
RETURN gds.util.asNode(node1).name AS actorName1,
    gds.util.asNode(node2).name AS actorName2,
    similarity
LIMIT 10
```


# Graph Machine Learning
GDS focuses on **offering managed pipelines for end-to-end ML** workflows. Data selection, feature engineering, data splitting, hyperparameter configuration, and training steps are coupled together within the pipeline object to track the end-to-end steps needed.

There are currently two supported types of ML pipelines:

- **Node Classification Pipelines**: Supervised binary and multi-class classification for nodes
- **Link Prediction Pipelines**: Supervised prediction for whether a relationship or "link" should exist between pairs of nodes

These pipelines have a `train` procedure that, once run, produces a trained model object. These trained model objects, in turn, have a `predict` procedure that can be used to produce predictions on the data.

## Node Classification

<img title="a title" alt="Alt text" src="./images/nc-pipeline-flow.png"  width="50%">

So at a high level, your workflow will look like the following:

- Project a graph and configure the pipeline (the order doesn’t matter).
- Execute the pipeline with a `train` command.
- Predict on a projected graph with the `predict` command. The predictions can then be written back to the database if desired using graph write operations.


## Link Prediction
GDS currently offers a **binary classifier where the target is a 0-1 indicator, 0 for no link, 1 for a link**. This type of link prediction works really well on an **undirected graph** where you are predicting one type of relationship between nodes of a single label, such as for social network and entity resolution problems.

<img title="a title" alt="Alt text" src="./images/link-prediction-flow.png"  width="50%">


# Path Finding



## Shortest Path with Cypher
### Unweighted Shortest Path
Cypher supports the calculation of the **shortest unweighted path** between a pair of nodes with the `shortestPath()` function.

In an `unweighted` path, the traversal of each **relationship has an identical cost**, so the shortest path between two nodes will always be the sum of the total relationships in a path between them.

The `shortestPath()` function **expects a definition of a Cypher path between an origin and destination node.** 

**The calculation is then made by traversing through the graph from both nodes, finding where the two paths meet in the middle. The traversal is `breadth first`, meaning that all relationships from a node will be followed before traversing further through the graph.**

**The overall shortest path is calculated as the smallest path length between the two nodes**, or in other words, **the smallest number of relationships in the path.**

```
MATCH (source:Airport {name:"Montana"}),
      (target:Airport {name:"Paris"}) //(1)
MATCH p = shortestPath((source)-[:HAS_ROUTE*1..10]->(target)) //<2>, (3)
RETURN p
```

The pattern defined in the query above instructs the algorithm to **follow** any `[:HAS_ROUTE]` **relationship type** in an **outgoing direction** from the source node **until** it reaches the **target**.

You can further **extend the types of relationships by using** the pipe operator `(|)` to add additional relationship types. For example, the below statement will follow either `[:HAS_ROUTE]`, `[:CAN_WALK_TO]` or `[:CAN_SWIM_TO]` relationships until it reaches the destination.

```
// example code - do not run
MATCH (source:Airport {name:"Montana"}),
      (target:Airport {name:"Paris"})
MATCH p=shortestPath((source)-[:HAS_ROUTE|CAN_WALK_TO|CAN_SWIM_TO*1..10]->(target))
RETURN p
```

The `allShortestPaths()` function works in the same way as the `shortestPath()` function, but instead of returning the first shortest path, the function **will return all paths that contain the shortest number of relationships**.

```
// example - do not run
MATCH (source:Airport {name:"Montana"}),
      (target:Airport {name:"Paris"})
MATCH p = allShortestPaths((source)-[:HAS_ROUTE*1..10]->(target))
RETURN p
```
## Shortest path GDS library

